In [ ]:
import json
import os
from collections import Counter
from nltk.translate.bleu_score import sentence_bleu

In [ ]:
PROJECT_PATH = ''#path to project
DATAPATH = ''#path to code

In [ ]:
FOLDER_NAME = 'tmcd_sparql_hirm_control_v5'
FILE_NAME = 'epoch_94_tm_0.85_t5_predictions.json'

### Чтение предсказания

In [ ]:
prediction_file = json.load(open(os.path.join(DATAPATH, FOLDER_NAME, FILE_NAME), 'r'))
prediction_list, true_list = prediction_file['predicted_queries'], prediction_file['true_queries']
prediction_file['test_metrics'], len(prediction_list), len(true_list)

### Exact Match

In [ ]:
hits = 0
for true, pred in zip(prediction_list, true_list):
    if true.lower() == pred.lower():
        hits += 1
    
exact_match = hits / len(true_list)
print('Exact Match: ', round(exact_match, 3))

### BLEU

In [ ]:
def calculate_bleu(pred_query, true_query):
    pred_tokens = pred_query.split()
    true_tokens = true_query.split()
    # TODO - set weights as average per compounds
    bleu = sentence_bleu([true_tokens], pred_tokens)
    return bleu

In [ ]:
total_bleu = 0
for true, pred in zip(prediction_list, true_list):
    total_bleu += calculate_bleu(pred.lower(), true.lower())
    
print('BLEU: ', round(total_bleu/len(true_list), 3))

### NGRAM F1

In [ ]:
def calculate_token_wise_f1(pred_query, true_query):
    pred_tokens = pred_query.split()
    true_tokens = true_query.split()
    
    common = Counter(pred_tokens) & Counter(true_tokens)
    num_same = sum(common.values())
    
    recall = num_same/len(true_tokens)
    precision = num_same/len(pred_tokens)
    f1 =  (2*precision*recall)/(recall+precision)
    return {
        "recall": recall,
        "precision": precision,
        "f1": f1
    }

In [ ]:
total_recall, total_precision, total_f1 = 0, 0, 0
for true, pred in zip(prediction_list, true_list):
    metrics_dict = calculate_token_wise_f1(pred.lower(), true.lower())
    
    total_recall += metrics_dict['recall']
    total_precision += metrics_dict['precision']
    total_f1 += metrics_dict['f1']
    
print('Average recall: ', round(total_recall/len(true_list), 3))
print('Average precision: ', round(total_precision/len(true_list), 3))
print('Average f1: ', round(total_f1/len(true_list), 3))

### Что за ошибки в тесте?